In [9]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 4.5 MB/s  0:00:02a 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] 1/2 [statsmodels]


In [6]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import acorr_ljungbox

In [13]:
def stationarity_check(ts):
    result = adfuller(ts)
    print(f"ADF Statistic : {result[0]}")
    print(f"p-value:{result[1]}")
    print(f"Critical values : {result[4]}")

def plot_acf_pacf(ts):
    fig,(ax1,ax2) = plt.subplots(1,2,figsize=(12,4))
    plot_acf(ts,ax=ax1,lags=20)
    plot_acf(ts,ax=ax2,lags=20)
    plt.show()

In [14]:
stock_symbol = "AAPL"
start_date = "2015-01-01"
end_date = "2023-01-01"
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)['Close']

/tmp/ipykernel_13811/1853554278.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(stock_symbol, start=start_date, end=end_date)['Close']
[*********************100%***********************]  1 of 1 completed
